# Martin's menace using genetic algorithms




In [ ]:
## Preamble

# %%
# Library Imports
import os
import sys
import pandas as pd
import datetime
import numpy as np
import re
import math

# Specific libraries
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from matplotlib import pyplot
from shapely.ops import transform, unary_union
from shapely.geometry import Polygon, MultiPoint, MultiPolygon
from shapely import affinity

from fastprogress import master_bar, progress_bar
from fastai.vision.core import PILImage
from PIL import Image

import io

from IPython.display import display, clear_output

import time

from functools import lru_cache

from descartes.patch import PolygonPatch

import itertools
import numpy as np
import tqdm

import numpy as np

In [ ]:
# %%
# Environment Variables
# Load environment variables, e.g., API keys, database URIs
# Assuming you have a .env file in your notebook directory or its parent
try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    print("dotenv not installed. Install it or manually set environment variables.")

DATA_PATH = Path(os.getenv("DATA_PATH", "data"))
# API_KEY = os.getenv("API_KEY")
# DATABASE_URI = os.getenv("DATABASE_URI")

In [ ]:
# %%
# Configuration
pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 2)

# Optional: Logging Configuration
import logging

logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(name)s - %(message)s"
)
# Disable logging for Pillow (or any other specific module)
logging.getLogger("PIL").setLevel(logging.CRITICAL)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

logger = logging.getLogger(__name__)

# %%
logger.info(f"Start {datetime.datetime.now()=}")

In [ ]:
from collections import defaultdict
from dataclasses import dataclass, field


@dataclass
class Piece:
    name: str
    pattern: np.ndarray = field(compare=True)

    COLORS = defaultdict(
        lambda: "gray", {"T": "blue", "s": "green", "r": "red", "f": "purple"}
    )

    def __init__(
        self, name: str, id: int, ascii_pattern: list[str] = None, pattern=None
    ):
        self.name = name
        self.id = id
        if pattern is None:
            self.pattern = np.array(
                [[(1 if c != " " else 0) for c in row] for row in ascii_pattern]
            )
        else:
            self.pattern = pattern

    def color(self):
        return self.COLORS[self.name]

    def flip(self):
        return Piece(self.name, self.id, pattern=np.flip(self.pattern, axis=0))

    def rot(self, degrees):
        assert degrees in [0, 90, 180, 270], f"{degrees=}"
        pattern = self.pattern
        while degrees > 0:
            pattern = np.rot90(pattern)
            degrees -= 90
        return Piece(self.name, self.id, pattern=pattern)

    def all_versions(self):
        bases = [self, self.flip()]
        versions = [
            base.rot(degrees) for base in bases for degrees in [0, 90, 180, 270]
        ]
        return list(sorted(set(versions)))

    def __lt__(self, other):
        for a, b in zip(self.pattern.ravel(), other.pattern.ravel()):
            if a < b:
                return True
            if a > b:
                return False
        return False  # equal

    def __eq__(self, other):
        return other.name == self.name and np.array_equal(self.pattern, other.pattern)

    def __hash__(self):
        return hash(self.pattern.tobytes()) + self.name.__hash__()


piece_T = Piece("T", id=1, ascii_pattern=["***", " * ", " * "])
piece_s = Piece("s", id=2, ascii_pattern=[" **", " * ", "** "])
piece_r = Piece("r", id=3, ascii_pattern=[" **", "** ", " * "])
piece_f = Piece("f", id=4, ascii_pattern=["* ", "**", "* ", "* "])

assert piece_T != piece_f
assert piece_T.rot(180) == piece_T.flip()
assert piece_f.rot(180) != piece_T.flip()

assert len(piece_T.all_versions()) == 4

pieces_dict = {piece.id: piece for piece in [piece_T, piece_s, piece_r, piece_f]}

In [ ]:
TARGET_WIDTH, TARGET_HEIGHT = 5.9, 4.95


# TODO cache rot, flip
class Board:
    def __init__(self, x=0, y=0, pattern=None):
        self.x = x
        self.y = y
        self.pattern = (
            piece_T.pattern.copy() * piece_T.id if pattern is None else pattern
        )

    @lru_cache
    def cost(self):
        all_pieces = []
        for piece, color in self.blocks():
            all_pieces.append(piece)
        full_polygon = unary_union(all_pieces)
        minimum_rotated_rectangle = full_polygon.minimum_rotated_rectangle

        # Extract the coordinates of the rectangle
        coords = list(minimum_rotated_rectangle.exterior.coords)

        # Calculate distances between the first three vertices
        # (adjacent vertices give width and height respectively)
        width = math.sqrt(
            (coords[0][0] - coords[1][0]) ** 2 + (coords[0][1] - coords[1][1]) ** 2
        )
        height = math.sqrt(
            (coords[1][0] - coords[2][0]) ** 2 + (coords[1][1] - coords[2][1]) ** 2
        )

        # Ensure width is the larger dimension and height is the smaller
        width, height = max(width, height), min(width, height)
        cost = (
            max(width - TARGET_WIDTH, 0) ** 2 + max(height - TARGET_HEIGHT, 0) ** 2
        ) ** 0.5
        return minimum_rotated_rectangle.area, (width, height), cost

    def blocks(self):
        for id in np.unique(self.pattern):
            blocks = []
            if id == 0:
                continue
            for i, row in enumerate(self.pattern):
                for j, cell in enumerate(row):
                    if cell == id:
                        blocks.append(
                            Polygon([(i, j), (i + 1, j), (i + 1, j + 1), (i, j + 1)])
                        )
            yield (MultiPolygon(blocks), pieces_dict[id].color())

    def plot(self, ax):
        all_pieces = []
        for piece, color in self.blocks():
            assert isinstance(piece, MultiPolygon)
            all_pieces.append(piece)
            for polygon in piece.geoms:
                ax.fill(*polygon.exterior.xy, fill=True, alpha=0.3, color=color)

        full_polygon = unary_union(all_pieces)
        convex_hull = full_polygon.convex_hull
        ax.fill(*convex_hull.exterior.xy, alpha=0.5, fill=False, color="black")
        ax.set_aspect("equal")
        ax.set_xlim(-1, self.pattern.shape[0] + 1)
        ax.set_xticks(range(-1, self.pattern.shape[0] + 1))
        ax.set_ylim(-1, self.pattern.shape[1] + 1)
        ax.set_yticks(range(-1, self.pattern.shape[1] + 1))

        bbox = full_polygon.minimum_rotated_rectangle
        ax.fill(*bbox.exterior.xy, alpha=0.5, fill=False, color="gray")
        area, measures, cost = self.cost()
        w, h = measures
        ax.set_title(f"{area=:.2f} {w=:.2f} {h=:.2f} {cost=:.2f}")

    def __repr__(self):
        line = ""  # f"{self.is_valid()=} {self.cost()=}"
        line += "\n" + "=" * (self.pattern.shape[1] + 2) + "\n"
        for row in self.pattern:
            line += "|" + "".join([str(cell) if cell else " " for cell in row]) + "|\n"
        line += "" + "=" * (self.pattern.shape[1] + 2) + "\n"
        return line

    def add(self, piece, flip, rotate_degrees, x, y):
        if flip:
            piece = piece.flip()
        if rotate_degrees:
            piece = piece.rot(rotate_degrees)

        while True:
            pattern = self.pattern.copy()

            if x < self.x:
                pattern = np.vstack(
                    (np.zeros((self.x - x, pattern.shape[1]), dtype=int), pattern)
                )
                b_x = x
            else:
                b_x = self.x

            if y < self.y:
                pattern = np.hstack(
                    (np.zeros((pattern.shape[0], self.y - y), dtype=int), pattern)
                )
                b_y = y
            else:
                b_y = self.y
            if x - b_x + piece.pattern.shape[0] > pattern.shape[0]:
                pattern = np.vstack(
                    (
                        pattern,
                        np.zeros(
                            (
                                x - b_x + piece.pattern.shape[0] - pattern.shape[0],
                                pattern.shape[1],
                            ),
                            dtype=int,
                        ),
                    )
                )
            if y - b_y + piece.pattern.shape[1] > pattern.shape[1]:
                pattern = np.hstack(
                    (
                        pattern,
                        np.zeros(
                            (
                                pattern.shape[0],
                                y - b_y + piece.pattern.shape[1] - pattern.shape[1],
                            ),
                            dtype=int,
                        ),
                    )
                )
            # print(f"{pattern=}")
            if np.any(
                (
                    pattern[
                        x - b_x : x - b_x + piece.pattern.shape[0],
                        y - b_y : y - b_y + piece.pattern.shape[1],
                    ]
                    != 0
                )
                & (piece.pattern != 0)
            ):
                x += 1  # correction
            else:
                pattern[
                    x - b_x : x - b_x + piece.pattern.shape[0],
                    y - b_y : y - b_y + piece.pattern.shape[1],
                ] += (
                    piece.pattern * piece.id
                )
                break

        return Board(b_x, b_y, pattern)


# board=Board().add(piece_f, *(True, 90, 2, 0))
# board=Board().add(piece_r, True, 180, -4, -1).add(piece_s, True, 270, -2, 0)
board = Board().add(piece_r, False, 270, 0, -2)
board

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
board.plot(ax)

In [ ]:
import array
import numpy

import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [ ]:
from operator import attrgetter

# based on SelTournament
def customSelTournamentNoDuplicates(individuals, k, tournsize, fit_attr="fitness"):
    """Select the best individual among *tournsize* randomly chosen
    individuals, *k* times. The list returned contains
    references to the input *individuals*.

    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :param tournsize: The number of individuals participating in each tournament.
    :param fit_attr: The attribute of individuals to use as selection criterion
    :returns: A list of selected individuals.

    This function uses the :func:`~random.choice` function from the python base
    :mod:`random` module.
    """
    chosen = set()
    non_duplicate = []
    for individual in individuals:
        if tuple(individual) in chosen:
            pass
        else:
            chosen.add(tuple(individual))
            non_duplicate.append(individual)

    individuals = non_duplicate
    chosen = []
    for i in range(k):
        aspirants = tools.selRandom(individuals, tournsize)
        chosen.append(max(aspirants, key=attrgetter(fit_attr)))
    return chosen

In [ ]:
from functools import lru_cache


toolbox = base.Toolbox()

MIN_X, MAX_X = -4, 4
MIN_Y, MAX_Y = -4, 4
pieces_to_position = [piece_f, piece_r, piece_s]


@lru_cache(2 * 4 * (MAX_X - MIN_X + 1) * (MAX_Y - MIN_Y + 1))
def build_board_from_tuple(individual):
    board = Board()
    assert len(individual) == 4 * 3
    for i in range(3):
        flip, rotation, x, y = individual[i * 4 : i * 4 + 4]
        piece = pieces_to_position[i]
        board = board.add(piece, *(flip, rotation, x, y))
    return board


def build_board_from_list(individual):
    return build_board_from_tuple(tuple(individual))


def evalBoard(individual):
    board = build_board_from_list(individual)
    area, measures, cost = board.cost()
    x, y = measures
    return (cost + (10 if x == int(x) else 0),)  # penalize orthogonal


def mutate(ind, pFlip=0.2, pRotate=0.2, pX=0.2, pY=0.2):
    mutant = toolbox.clone(ind)
    gene = random.randint(0, 2) * 4
    assert len(mutant) == 4 * 3
    assert len(mutant) > gene + 2, f"{mutant=} {gene=}"
    if random.random() < pFlip:  # flip
        mutant[gene + 0] = not mutant[gene + 0]
    if random.random() < pRotate:  # rotation
        mutant[gene + 1] = (mutant[gene + 1] + random.choice([-90, 90])) % 360
    if random.random() < pX:  # x
        mutant[gene + 2] = toolbox.attr_x()
    if random.random() < pY:  # y
        mutant[gene + 3] = toolbox.attr_y()
    assert mutant[1] in [0, 90, 180, 270], f"{ind=} {mutant=} {gene=}"
    assert mutant[1 + 4] in [0, 90, 180, 270]
    assert mutant[1 + 4 * 2] in [0, 90, 180, 270]
    return (mutant,)


def crossover(ind1, ind2):
    child1, child2 = toolbox.clone(ind1), toolbox.clone(ind2)
    gene = random.randint(0, 2) * 4
    assert len(ind1) == 4 * 3
    assert len(ind2) == 4 * 3
    for i in range(len(ind1)):
        if gene <= i < gene + 4:
            child1[i], child2[i] = ind2[i], ind1[i]
    # print(f"{gene=} {child1=} {child2=}")
    assert child1[1] in [0, 90, 180, 270]
    assert child1[1 + 4] in [0, 90, 180, 270]
    assert child1[1 + 4 * 2] in [0, 90, 180, 270]
    assert child2[1] in [0, 90, 180, 270]
    assert child2[1 + 4] in [0, 90, 180, 270]
    assert child2[1 + 4 * 2] in [0, 90, 180, 270]
    return child1, child2


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


toolbox.register("attr_flip", random.choice, [True, False])
# toolbox.register("attr_flip", random.choice, [False])
toolbox.register("attr_rotate_degrees", random.choice, [0, 90, 180, 270])
toolbox.register("attr_x", random.randint, MIN_X, MAX_X)
toolbox.register("attr_y", random.randint, MIN_Y, MAX_Y)
toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (toolbox.attr_flip, toolbox.attr_rotate_degrees, toolbox.attr_x, toolbox.attr_y),
    n=3,
)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalBoard)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate, pFlip=0.2, pRotate=0.2, pX=0.2, pY=0.2)
toolbox.register("select", customSelTournamentNoDuplicates, tournsize=3)

POPULATION = 20000
pop = toolbox.population(n=POPULATION)
hof = tools.HallOfFame(4)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, log = algorithms.eaSimple(
    pop,
    toolbox,
    cxpb=0.2,
    mutpb=0.2,
    ngen=25,
    stats=stats,
    halloffame=hof,
    verbose=True,
)

# pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=POPULATION//10, lambda_=POPULATION, cxpb=0.5, mutpb=0.5, ngen=25,
#                                 stats=stats, halloffame=hof, verbose=True)

print(hof[0])

for best in hof:
    fig, ax = plt.subplots(figsize=(4, 4))
    build_board_from_list(best).plot(ax)